In [ ]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# path to the sqlite file
db_path = "data/processed/NVDA_proc.sqlite"

# connect and discover table name (pick a table containing 'nvda' if present, else first table)
conn = sqlite3.connect(db_path)
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
if tables.empty:
    conn.close()
    raise RuntimeError(f"No tables found in {db_path}")

table_names = tables['name'].astype(str).tolist()
table = next((t for t in table_names if 'nvda' in t.lower()), table_names[0])

# load the table
df = pd.read_sql_query(f'SELECT * FROM "{table}"', conn)
conn.close()

# try to identify a date column and set it as index
date_cols = [c for c in df.columns if c.lower() in ('date', 'datetime', 'time', 'timestamp', 'index')]
if date_cols:
    date_col = date_cols[0]
    df[date_col] = pd.to_datetime(df[date_col], errors='coerce')
    df = df.dropna(subset=[date_col]).set_index(date_col)
else:
    # if no obvious date column, assume the first column is a datetime-like index if possible
    try:
        df.index = pd.to_datetime(df.iloc[:, 0])
        df = df.iloc[:, 1:]
    except Exception:
        # keep as-is
        pass

# ensure data sorted by index (time)
df = df.sort_index()

# ensure we have a Close price column
close_col_candidates = [c for c in df.columns if c.lower() in ('close', 'adj_close', 'close_price', 'closepx')]
if not close_col_candidates:
    raise RuntimeError("No 'Close' column found in the table. Columns: " + ", ".join(df.columns))
close_col = close_col_candidates[0]
df = df.rename(columns={close_col: 'Close'})

# compute moving averages
df['MA50'] = df['Close'].rolling(window=50, min_periods=1).mean()
df['MA200'] = df['Close'].rolling(window=200, min_periods=1).mean()

# generate crossover signals:
# buy (1) when MA50 crosses above MA200; sell (-1) when MA50 crosses below MA200; otherwise 0
diff = df['MA50'] - df['MA200']
df['signal'] = 0
buy_mask = (diff.shift(1) < 0) & (diff > 0)
sell_mask = (diff.shift(1) > 0) & (diff < 0)
df.loc[buy_mask, 'signal'] = 1
df.loc[sell_mask, 'signal'] = -1

# position: 1 when MA50 > MA200 (long), 0 otherwise
df['position'] = (df['MA50'] > df['MA200']).astype(int)

# optional: compute strategy returns (simple example, assumes entering/exiting at Close)
df['market_return'] = df['Close'].pct_change().fillna(0)
# shift position so that today's position reflects entry at close after signal (simple backtest assumption)
df['strategy_return'] = df['position'].shift(1).fillna(0) * df['market_return']
df['strategy_eq'] = (1 + df['strategy_return']).cumprod()

# show last rows with signals
print(df[['Close', 'MA50', 'MA200', 'signal', 'position']].tail(20))

# plot Close and MAs and mark buy/sell signals
plt.figure(figsize=(12, 6))
plt.plot(df.index, df['Close'], label='Close', color='black', linewidth=1)
plt.plot(df.index, df['MA50'], label='MA50', color='blue', linewidth=1)
plt.plot(df.index, df['MA200'], label='MA200', color='orange', linewidth=1)

# plot buy/sell markers
buys = df[df['signal'] == 1]
sells = df[df['signal'] == -1]
plt.scatter(buys.index, buys['Close'], marker='^', color='green', label='Buy', zorder=5)
plt.scatter(sells.index, sells['Close'], marker='v', color='red', label='Sell', zorder=5)

plt.legend()
plt.title(f"Trend-following signals from {table}")
plt.xlabel("Date")
plt.ylabel("Price")
plt.grid(True)
plt.show()